<a href="https://colab.research.google.com/github/sayakpaul/Consistency-Training-with-Supervision/blob/main/CIFAR_10C_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
# All model weights
!wget https://git.io/JOKI9 -O consistency_training_model_weights.zip

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm
import numpy as np

## Define Hyperparameters

In [ ]:
AUTO = tf.data.AUTOTUNE
DATASET_NAME = "cifar10_corrupted"
BATCH_SIZE = 128
IMAGE_SIZE = 72

In [ ]:
VERSIONS = [
    "brightness_5",
    "contrast_5",
    "defocus_blur_5",
    "elastic_5",
    "fog_5",
    "frost_5",
    "frosted_glass_blur_5",
    "gaussian_blur_5",
    "gaussian_noise_5",
    "impulse_noise_5",
    "jpeg_compression_5",
    "motion_blur_5",
    "pixelate_5",
    "saturate_5",
    "shot_noise_5",
    "snow_5",
    "spatter_5",
    "speckle_noise_5",
    "zoom_blur_5"
]

print(f"Total sub-versions of the CIFAR10-C dataset: {len(VERSIONS)}")

Total sub-versions of the CIFAR10-C dataset: 19


## Utilities

In [ ]:
def prepare_dataset(ds):
    ds = (ds
          .batch(BATCH_SIZE)
          .map(lambda x, y: (tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE)), y), 
               num_parallel_calls=AUTO)
          .prefetch(AUTO)
    )
    return ds

In [ ]:
def get_training_model(num_classes=10):
    resnet50_v2 = tf.keras.applications.ResNet50V2(
        weights=None,
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    )
    model = tf.keras.Sequential(
        [
            layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3)),
            layers.experimental.preprocessing.Rescaling(scale=1.0 / 127.5, offset=-1),
            resnet50_v2,
            layers.GlobalAveragePooling2D(),
            layers.Dense(num_classes)
        ]
    )
    return model

In [ ]:
def evaluate_model(model):
    acc_dict = {}
    for version in tqdm(VERSIONS):
        print(f"Processing {version}")
        dataset_fullname = DATASET_NAME + "/" + version
        loaded_ds = tfds.load(
            dataset_fullname,
            split="test",
            as_supervised=True
        )
        loaded_ds = prepare_dataset(loaded_ds)
        _, acc = model.evaluate(loaded_ds, verbose=0)
        print(f"Test accuracy on {version}: {acc*100}%")
        acc_dict[version] = acc*100
        
    return acc_dict, np.mean(list(acc_dict.values()))

## Evaluation

### SWA

In [ ]:
# Evaluate teacher model trained with SWA
teacher_model_swa = get_training_model()
teacher_model_swa.load_weights("teacher_model_swa.h5")
teacher_model_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_swa)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:17<05:20, 17.80s/it]

Test accuracy on brightness_5: 76.05999708175659%
Processing contrast_5


 11%|█         | 2/19 [00:19<02:22,  8.40s/it]

...

Test accuracy on spatter_5: 64.16000127792358%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:49<00:01,  1.86s/it]

Test accuracy on speckle_noise_5: 45.62999904155731%
Processing zoom_blur_5


100%|██████████| 19/19 [00:51<00:00,  2.70s/it]

Test accuracy on zoom_blur_5: 77.49000191688538%
Mean Top-1 Accuracy: 59.695262579541456%


In [ ]:
# Evaluate the corresponding student model
student_noisy_swa = get_training_model()
student_noisy_swa.load_weights("student_noisy_swa.h5")
student_noisy_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_swa)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:02<00:51,  2.85s/it]

Test accuracy on brightness_5: 82.91000127792358%
Processing contrast_5


 11%|█         | 2/19 [00:04<00:37,  2.22s/it]

...

Test accuracy on spatter_5: 70.31999826431274%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:32<00:01,  1.76s/it]

Test accuracy on speckle_noise_5: 53.46999764442444%
Processing zoom_blur_5


100%|██████████| 19/19 [00:34<00:00,  1.82s/it]

Test accuracy on zoom_blur_5: 79.67000007629395%
Mean Top-1 Accuracy: 62.80894765728399%


### MA

In [ ]:
# Evaluate teacher model trained with MA
teacher_model_ma = get_training_model()
teacher_model_ma.load_weights("teacher_model_ma.h5")
teacher_model_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_ma)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:02<00:50,  2.83s/it]

Test accuracy on brightness_5: 73.14000129699707%
Processing contrast_5


 11%|█         | 2/19 [00:04<00:37,  2.21s/it]

...

Test accuracy on spatter_5: 63.8700008392334%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:32<00:01,  1.75s/it]

Test accuracy on speckle_noise_5: 49.75000023841858%
Processing zoom_blur_5


100%|██████████| 19/19 [00:34<00:00,  1.81s/it]

Test accuracy on zoom_blur_5: 78.32000255584717%
Mean Top-1 Accuracy: 59.89315792133934%


In [ ]:
# Evaluate the corresponding student model
student_noisy_ma = get_training_model()
student_noisy_ma.load_weights("student_noisy_ma.h5")
student_noisy_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_ma)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:02<00:51,  2.84s/it]

Test accuracy on brightness_5: 81.80999755859375%
Processing contrast_5


 11%|█         | 2/19 [00:04<00:37,  2.20s/it]

...

Test accuracy on spatter_5: 70.24000287055969%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:32<00:01,  1.75s/it]

Test accuracy on speckle_noise_5: 52.27000117301941%
Processing zoom_blur_5


100%|██████████| 19/19 [00:34<00:00,  1.81s/it]

Test accuracy on zoom_blur_5: 77.46000289916992%
Mean Top-1 Accuracy: 61.863684340527186%


### Regular

In [ ]:
# Evaluate teacher model 
teacher_model = get_training_model()
teacher_model.load_weights("teacher_model.h5")
teacher_model.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<00:55,  3.08s/it]

Test accuracy on brightness_5: 75.52000284194946%
Processing contrast_5


 11%|█         | 2/19 [00:04<00:38,  2.29s/it]

...

Test accuracy on spatter_5: 63.81999850273132%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:32<00:01,  1.74s/it]

Test accuracy on speckle_noise_5: 49.059998989105225%
Processing zoom_blur_5


100%|██████████| 19/19 [00:34<00:00,  1.82s/it]

Test accuracy on zoom_blur_5: 77.02000141143799%
Mean Top-1 Accuracy: 60.51105205949984%


In [ ]:
# Evaluate the corresponding student model
student_noisy = get_training_model()
student_noisy.load_weights("student_noisy.h5")
student_noisy.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:02<00:52,  2.92s/it]

Test accuracy on brightness_5: 79.80999946594238%
Processing contrast_5


 11%|█         | 2/19 [00:04<00:37,  2.23s/it]

...

Test accuracy on spatter_5: 65.49000144004822%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:32<00:01,  1.75s/it]

Test accuracy on speckle_noise_5: 48.96999895572662%
Processing zoom_blur_5


100%|██████████| 19/19 [00:34<00:00,  1.81s/it]

Test accuracy on zoom_blur_5: 75.85999965667725%
Mean Top-1 Accuracy: 58.98000041120931%


## Evaluate on CIFAR-10 Test Set

In [ ]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = prepare_dataset(test_ds)

In [ ]:
# Evaluate teacher model trained with SWA
_, test_acc = teacher_model_swa.evaluate(test_ds, verbose=0)
print("Test accuracy with SWA teacher: {:.2f}%".format(test_acc * 100))

# Evaluate the corresponding student
_, test_acc = student_noisy_swa.evaluate(test_ds, verbose=0)
print("Test accuracy with noisy SWA student: {:.2f}%".format(test_acc * 100))

Test accuracy with SWA teacher: 84.82%
Test accuracy with noisy SWA student: 85.24%


In [ ]:
# Evaluate teacher model trained with MA
_, test_acc = teacher_model_ma.evaluate(test_ds, verbose=0)
print("Test accuracy with MA teacher: {:.2f}%".format(test_acc * 100))

# Evaluate the corresponding student
_, test_acc = student_noisy_ma.evaluate(test_ds, verbose=0)
print("Test accuracy with noisy MA student: {:.2f}%".format(test_acc * 100))

Test accuracy with MA teacher: 83.88%
Test accuracy with noisy MA student: 84.42%


In [ ]:
# Evaluate regular teacher model
_, test_acc = teacher_model.evaluate(test_ds, verbose=0)
print("Test accuracy with regular teacher: {:.2f}%".format(test_acc * 100))

# Evaluate the corresponding student
_, test_acc = student_noisy.evaluate(test_ds, verbose=0)
print("Test accuracy with regular noisy student: {:.2f}%".format(test_acc * 100))

Test accuracy with regular teacher: 83.20%
Test accuracy with regular noisy student: 82.16%
